In [1]:
'''
dictionary = something belongs to something else, is kwargs, or has certain order
list = collection of several parallel operations

blocks -> first level keys (THESE HAVE SET ORDER)
Below this ^^^ -> The order you provide the code will matter
    - dict = matters
    - list = does not matter

'''
import sys
import os
#print(os.getcwd())
sys.path.append("../")
from sklearn.preprocessing import *
from sklearn.model_selection import *
from scipy.stats import *
import pandas as pd
from sklearn.metrics import *
from wmp_ml.static.splitting import *
from wmp_ml.static.preprocessing import *
from wmp_ml.static.feature_engineering import *
from wmp_ml.static.eda import *
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from wmp_ml.dag.layers import DagLayer

kwargs = \
{   
     'modeling': { 
                    'models': DagLayer({'LOG': {LogisticRegression: None}, 
                                       'GNB': {GaussianNB: None},
                                       'RF':{RandomForestClassifier: {'n_estimators': 10}},
                                       'MLP': {MLPClassifier: {'activation': 'tanh'}},
                                       'SVM': {SVC: {'kernel': 'linear'}},
                                       'KNN': {KNeighborsClassifier: {'n_neighbors':3 }}}),              
                 },
 
     'data_sources': [DagLayer({'https://raw.githubusercontent.com/SamShowalter/WMP_training/01_EDA/data/IBM_Employee_Attrition.csv': {pd.read_csv:None}})],

#Expects flat dataset
    'splitting': [
                        DagLayer({'split': {auto_train_test_split  : {'target': "Attrition",
                                                                        'test_ratio': 0.2,
                                                                         'random_state': 42}}}),
                 ],
 
     
     #Expects flat dataset
        'preprocessing': { 
                        
                        'cleansing': [   
                                              DagLayer({'winsorize': {wmp_winsorize: {'limits' : [0.05, 0.05],
                                                                                      'col_names': ['MonthlyRate',
                                                                                                    'NumCompaniesWorked',
                                                                                                   'PercentSalaryHike']}}}),
                                              DagLayer({'median_impute': {wmp_impute : {'method': "median"}}})
                                     ]
                        
                     },
    #Expects flat dataset
    'feature_engineering': {
                            'transformations': DagLayer({
                                    ('Department', 'EducationField', 'JobRole', "MaritalStatus"): {pd.get_dummies: None},
                                    'BusinessTravel': {create_ordinal_df: {'ordinal_dict':{"Non-Travel" :0, 
                                                                                            "Travel_Rarely":1, 
                                                                                            "Travel_Frequently": 2},
                                                                          'tag': None}},
                                    ('Attrition', 'Gender', 'OverTime') : {convert_boolean_df: 
                                                                           {'boolean_names_and_values':
                                                                            {"Attrition": ["Yes", "No"],
                                                                            "Gender": ["Male", "Female"],
                                                                            "OverTime": ["Yes", "No"]}, 
                                                                            'tag' : None}},
                                    ('Age', 'DistanceFromHome', "EnvironmentSatisfaction",
                                    'JobInvolvement', 'JobSatisfaction', "MonthlyIncome",
                                    'NumCompaniesWorked', 'PercentSalaryHike', "PerformanceRating",
                                    'RelationshipSatisfaction', 'TotalWorkingYears',
                                    'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
                                    'YearsInCurrentRole', 'YearsSinceLastPromotion',
                                    'YearsWithCurrManager') : {normalize_values: {'tag': None}},
                                    'MonthlyRate': {wmp_linear_transformation: {'method': boxcox},
                                                    normalize_values: {'tag': None}},
                               })
                            
                           },
 
#      #Expects predictions
#      'evaluation':{'metrics' : DagLayer({'acc': {accuracy_score: None}, 
#                                         'f1': {f1_score: None}, 
#                                         'precision': {precision_score: None},
#                                         'conf_matrix': {confusion_matrix: None}, 
#                                         'class_report': {classification_report: None},
#                                         'recall': {recall_score :None}})
#                   }
   
}

In [2]:
import pandas as pd
import os
import sys
sys.path.append("../wmp_ml/static")
sys.path.append("../wmp_ml/dag")

In [3]:
from generator import DagGenerator
from datetime import datetime


In [4]:
dag_args = { 
                'dag_name': "WMP_ML_test_creation_full",
                'dag':      {
                                'owner': 'Sam Showalter',
                                'email': ['sshowalter@wmp.com'],
                                'op_args':{},
                                'op_kwargs': {},
                                'params': {}
                            },
                'config' : kwargs
}

In [5]:
dg = DagGenerator(dag_args)
dg.generate_file()

0 ['data_sources'] ds
2.0 ['preprocessing', 'cleansing'] p_cleansing_l0
2.1 ['preprocessing', 'cleansing'] p_cleansing_l1
3 ['feature_engineering', 'transformations'] fe_transformations
4 ['splitting'] s
5 ['modeling', 'models'] m_models
/home/sshowalter/airflow/dags/WMP_ML_test_creation_full_Sam-Showalter_10-17-2019--09.17.26.py
